<a href="https://colab.research.google.com/github/5eunji/Final-project-G3/blob/main/TTS_App(2)_%EB%8C%80%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **While-Listening Activity: Dialogue**
## 2. Gradio TTS App(대화형)

In [ ]:
!pip install gradio
!pip install gtts

import gradio as gr
from gtts import gTTS
import tempfile
import os

# Define the dialogue between Juliet and Romeo
dialogues = [
    ("Juliet", "Oh, Romeo, Romeo! Where are you, Romeo?"),
    ("Romeo", "Juliet, I am here, standing below your window."),
    ("Juliet", "Why must our families hate each other? Why can’t we be together?"),
    ("Romeo", "I don’t care about my family’s name. I only care about you, Juliet."),
    ("Juliet", "If you truly love me, promise to stay with me forever."),
    ("Romeo", "I promise, my love. I will always be with you, no matter what."),
    ("Juliet", "Then let’s marry in secret and prove that love can conquer anything.")
]

# Function to generate TTS for the entire dialogue and save as separate files
def generate_full_dialogue():
    combined_audio_files = []
    for i, (speaker, text) in enumerate(dialogues):
        # Different voices for Juliet and Romeo
        tld = 'co.uk' if speaker == "Juliet" else 'com'
        tts = gTTS(text=text, lang='en', tld=tld, slow=False)

        # Save audio to a temporary file
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        tts.save(temp_file.name)
        combined_audio_files.append((speaker, temp_file.name))

    return combined_audio_files

# Combine all dialogue files into one audio file (optional)
def combine_audio_files(audio_files):
    from pydub import AudioSegment
    combined_audio = AudioSegment.empty()

    for speaker, file in audio_files:
        audio = AudioSegment.from_file(file)
        combined_audio += audio

    # Save combined audio
    combined_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    combined_audio.export(combined_file.name, format="mp3")
    return combined_file.name

# Gradio interface for single-line playback and combined playback
def get_single_audio(selected_line):
    speaker, text = dialogues[int(selected_line.split(".")[0]) - 1]
    tld = 'co.uk' if speaker == "Juliet" else 'com'
    tts = gTTS(text=text, lang='en', tld=tld, slow=False)

    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    tts.save(temp_file.name)
    return temp_file.name, f"Speaker: {speaker}\n{text}"

# Gradio interface
with gr.Blocks() as demo:
    with gr.Row():
        gr.Markdown("## Romeo and Juliet Dialogue TTS Generator")

    # Single-line TTS playback
    with gr.Row():
        dropdown_dialogues = gr.Dropdown(
            choices=[f"{i + 1}. {dialogues[i][0]}: {dialogues[i][1][:50]}..." for i in range(len(dialogues))],
            label="Select Dialogue Line"
        )
        single_line_button = gr.Button("Generate TTS for Selected Line")
        single_line_audio = gr.Audio(type="filepath", label="Single Line Audio")
        displayed_text = gr.Textbox(label="Selected Dialogue", interactive=False)

        single_line_button.click(get_single_audio, inputs=[dropdown_dialogues], outputs=[single_line_audio, displayed_text])

    # Full dialogue playback
    with gr.Row():
        generate_full_button = gr.Button("Generate Full Dialogue Audio")
        full_audio_output = gr.Audio(type="filepath", label="Full Dialogue Audio")

        def generate_and_combine():
            audio_files = generate_full_dialogue()
            combined_audio_path = combine_audio_files(audio_files)
            return combined_audio_path

        generate_full_button.click(generate_and_combine, outputs=[full_audio_output])

demo.launch()
